In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\yboul\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
# Define url  and set up config splinter to the site 
#Create a function that takes the url and return the soup 
def create_soup(url):
    browser.visit(url)
    # Create BeautifulSoup object; parse with 'html.parser'
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    return soup


##### NASA Mars News
    - Scrape the Mars News Site and collect the latest News Title and Paragraph Text.


In [4]:
#Define the url to visit and call the function create_soup
url = 'https://mars.nasa.gov/news/'
soup = create_soup(url)

In [5]:
# grab the title
title = soup.find("li", class_="slide").find('div' , class_="content_title").text
news_p = soup.find("li", class_="slide").find('div', class_="article_teaser_body").text

print("Title: ",title)
print("Paragraph: ",news_p)

Title:  NASA's Mars Helicopter to Make First Flight Attempt Sunday
Paragraph:  The small rotorcraft’s “Wright brothers moment” is two Mars days away.



##### JPL Mars Space Images - Featured Image
    - Visit the url for the Featured Space Image site https://spaceimages-mars.com/
    - Find the image url to the full size .jpg image.


In [6]:
# Define url  and call create_soup function to return soup 
featured_url = 'https://spaceimages-mars.com/'
soup = create_soup(featured_url)

In [7]:
# Create BeautifulSoup object; parse with 'html.parser' to the second site
try:
    target = 'button[class="btn btn-outline-light"]'
    browser.find_by_tag(target).click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    image_src = soup.find('img', class_="fancybox-image")['src']
except:
    print('can\'t find the image')
featured_image_url = featured_url + image_src
print(featured_image_url)


https://spaceimages-mars.com/image/featured/mars3.jpg



##### Mars Facts
        - Visit the Mars Facts webpage https://space-facts.com/mars/ and use Pandas to scrape the table 
          containing facts about the planet including Diameter, Mass, etc.
        - Use Pandas to convert the data to a HTML table string.


In [8]:
# define url to visit
mars_facts_url = 'https://space-facts.com/mars/'
# using pandas read html
tables = pd.read_html(mars_facts_url)
# find total no. of tables in the list
len(tables)

3

In [9]:
# check for the table in iterest and select that table
mars_fact_df = tables[0]
mars_fact_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [10]:
# add column names ['Description' , 'Mars']
mars_fact_df.columns = ['Description' , 'Mars']

# set Description as a index
mars_fact_df.set_index('Description' , drop=True , inplace= True)
mars_fact_df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [11]:
# Save the table in HTML format
table = mars_fact_df.to_html('table.html')



##### Mars Hemispheres
          - Visit the astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
          - get image url to the full resolution image.


In [12]:
# Define the url and call create_soup function to return soup object
mars_hem_url = 'https://marshemispheres.com/'
soup = create_soup(mars_hem_url)

# Find the container that holds all the images in the main page
container = soup.find_all('div' , class_="item")

# Initialize the list
hemisphere_image_urls =[]

for content in container:    
    # use splinter to navigate to the page to get the image file 
    # Note: splinter uses main browser not the cotainer so be specific
    h3 = content.find('h3').text
    
    browser.find_by_text(str(h3)).click()
    
    # get html of rendered page and create a soup
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')   
    
    # Get the title and the image link
    title_hem = soup.find('h2' , class_="title").text
    
    image = mars_hem_url + soup.find('img', class_="wide-image")['src']
    #print(image)
    
    #create a dictionary
    hemisphere_image_urls.append({
                            'title' :title_hem,'img_url':image
                             })

    # go back to the original page to grab more data
    browser.back()
    
    

print(hemisphere_image_urls)


[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]


In [13]:
# Close the browser
browser.quit()
    